## Instalación de librerías

In [1]:
pip install transformers

     |████████████████████████████████| 4.0 MB 5.8 MB/s 
     |████████████████████████████████| 77 kB 8.6 MB/s 
     |████████████████████████████████| 895 kB 55.0 MB/s 
     |████████████████████████████████| 596 kB 52.5 MB/s 
     |████████████████████████████████| 6.6 MB 43.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
pip install colorama

### Cargando librerías

In [3]:
import pandas as pd
import numpy as np

import colorama
from colorama import Fore

import sklearn
from sklearn.model_selection import train_test_split

import tensorflow
import keras
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import datasets, layers, models

from transformers import pipeline, set_seed, BertTokenizer, TFBertModel
from transformers import AutoTokenizer, AutoModelForTokenClassification

In [4]:
from google.colab import drive 
drive.mount('/content/drive')
data_df = pd.read_csv('drive/My Drive/Colab_Notebooks/gap-development.tsv', delimiter='\t')

Mounted at /content/drive


In [5]:
data_df.head()

,ID,Text,Pronoun,Pronoun-offset,A,A-offset,A-coref,B,B-offset,B-coref,URL
0,development-1,Zoe Telford -- played the police officer girlf...,her,274,Cheryl Cassidy,191,True,Pauline,207,False,http://en.wikipedia.org/wiki/List_of_Teachers_...
1,development-2,"He grew up in Evanston, Illinois the second ol...",His,284,MacKenzie,228,True,Bernard Leach,251,False,http://en.wikipedia.org/wiki/Warren_MacKenzie
2,development-3,"He had been reelected to Congress, but resigne...",his,265,Angeloz,173,False,De la Sota,246,True,http://en.wikipedia.org/wiki/Jos%C3%A9_Manuel_...
3,development-4,The current members of Crime have also perform...,his,321,Hell,174,False,Henry Rosenthal,336,True,http://en.wikipedia.org/wiki/Crime_(band)
4,development-5,Her Santa Fe Opera debut in 2005 was as Nuria ...,She,437,Kitty Oppenheimer,219,False,Rivera,294,True,http://en.wikipedia.org/wiki/Jessica_Rivera


In [6]:
def gap_printer2(data_df_row):
        
    text   = data_df_row["Text"]
    word_A = data_df_row["A"]
    word_B = data_df_row["B"]
    
    pronoun       = data_df_row["Pronoun"]
    pronoun_begin = data_df_row["Pronoun-offset"]
    pronoun_end   = pronoun_begin + len(pronoun)
    
    word_A_begin = data_df_row["A-offset"]
    word_A_end   = data_df_row["A-offset"] + len(word_A)
    word_B_begin = data_df_row["B-offset"]
    word_B_end   = data_df_row["B-offset"] + len(word_B)
    
    text_c = text.replace(word_A, " {} ")
    text_c = text.replace(word_B, " {} ")
    text_c = text.replace(pronoun, " {} ")
    
    word_boundaries = np.sort([word_A_begin, word_A_end, pronoun_begin, pronoun_end, word_B_begin, word_B_end])
    word_boundaries = list(zip(word_boundaries[::2], word_boundaries[1::2]))
    
    P1 = [0,word_boundaries[0][0]]
    P2 = [word_boundaries[0][1],word_boundaries[1][0]]
    P3 = [word_boundaries[1][1],word_boundaries[2][0]]
    P4 = [word_boundaries[2][1],len(text)]

    text_f = text[P1[0]:P1[1]] + "{}" + text[P2[0]:P2[1]] +  "{}" + text[P3[0]:P3[1]] + "{}" + text[P4[0]:P4[1]]
 
    print(text_f.format( Fore.BLUE  + text[word_boundaries[0][0]:word_boundaries[0][1]]  + Fore.BLACK,
                         Fore.BLUE  + text[word_boundaries[1][0]:word_boundaries[1][1]] + Fore.BLACK,
                         Fore.BLUE  + text[word_boundaries[2][0]:word_boundaries[2][1]]  + Fore.BLACK))

In [7]:
gap_printer2(data_df.loc[0])

Zoe Telford -- played the police officer girlfriend of Simon, Maggie. Dumped by Simon in the final episode of series 1, after he slept with Jenny, and is not seen again. Phoebe Thomas played Cheryl Cassidy, Pauline's friend and also a year 11 pupil in Simon's class. Dumped her boyfriend following Simon's advice after he wouldn't have sex with her but later realised this was due to him catching crabs off her friend Pauline.


In [8]:
url = data_df["URL"][1]
url

'http://en.wikipedia.org/wiki/Warren_MacKenzie'

In [9]:
text = data_df["Text"][1]
text

'He grew up in Evanston, Illinois the second oldest of five children including his brothers, Fred and Gordon and sisters, Marge (Peppy) and Marilyn. His high school days were spent at New Trier High School in Winnetka, Illinois. MacKenzie studied with Bernard Leach from 1949 to 1952. His simple, wheel-thrown functional pottery is heavily influenced by the oriental aesthetic of Shoji Hamada and Kanjiro Kawai.'

In [10]:
pronoun = data_df["Pronoun"][1]

In [11]:
pronoun_offset = data_df["Pronoun-offset"][1]
pronoun_offset

284

In [12]:
text[pronoun_offset:pronoun_offset+len(pronoun)]

'His'

In [13]:
A = data_df["A"][1]
A_offset = data_df["A-offset"][1]
A, A_offset

('MacKenzie', 228)

In [14]:
text[A_offset:A_offset+len(A)]

'MacKenzie'

In [15]:
B = data_df["B"][1]
B_offset = data_df["B-offset"][1]
B, B_offset

('Bernard Leach', 251)

In [16]:
text[B_offset:B_offset+len(B)]

'Bernard Leach'

In [17]:
x_data = data_df[["Text", "Pronoun", "Pronoun-offset", "A", "A-offset", "B", "B-offset"]]
y_data = data_df[["A-coref", "B-coref"]]

In [18]:
x_train = x_data.loc[:1500,:]
y_train = y_data.loc[:1500,:]
x_test = x_data.loc[1500:,:]
x_test = x_test.reset_index()
y_test = y_data.loc[1500:,:]
y_test = y_test.reset_index()

In [19]:
x_train

,Text,Pronoun,Pronoun-offset,A,A-offset,B,B-offset
0,Zoe Telford -- played the police officer girlf...,her,274,Cheryl Cassidy,191,Pauline,207
1,"He grew up in Evanston, Illinois the second ol...",His,284,MacKenzie,228,Bernard Leach,251
2,"He had been reelected to Congress, but resigne...",his,265,Angeloz,173,De la Sota,246
3,The current members of Crime have also perform...,his,321,Hell,174,Henry Rosenthal,336
4,Her Santa Fe Opera debut in 2005 was as Nuria ...,She,437,Kitty Oppenheimer,219,Rivera,294
...,...,...,...,...,...,...,...
1496,"In 2014, Rockov also signed his first professi...",he,282,Nemanja Buzad*ija,395,Rockov,446
1497,Mr Telford was of the most genial disposition ...,him,209,George Turnbull,158,Telford,186
1498,"In 1865, the theatre was renovated and named t...",He,306,Henry Byron,184,Squire Bancroft,289
1499,As well as similarities as regards the type of...,his,399,Don Quixote,303,Cervantes,518


In [20]:
y_train

,A-coref,B-coref
0,True,False
1,True,False
2,False,True
3,False,True
4,False,True
...,...,...
1496,False,True
1497,True,False
1498,False,True
1499,False,False


Se renombran etiquetas de acuerdo con las siguientes convenciones:

-  0 pronombre no se refiere a nada
-  1 pronombre se refiere a columna A
-  2 pronombre se refiere a columna B
-  3 pronombre se refiere a columna A y B.

In [21]:
labels=y_train

In [22]:
test_labels=y_test

In [23]:
def calc_new_col(row):
    if (row['A-coref'] == False) & (row['B-coref'] == False):
        return 0
    elif (row['A-coref'] == True) & (row['B-coref'] == False):
        return 1
    elif (row['A-coref'] == False) & (row['B-coref'] == True):
        return 2
    else :
        return 3

labels["label"] = labels.apply(calc_new_col, axis=1)
# axis=1 makes sure that function is applied to each row

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [24]:
test_labels["label"] = test_labels.apply(calc_new_col, axis=1)

In [25]:
labels=labels['label']
test_labels = test_labels['label']

Estandarizando cadena de caracteres en minúsculas para evitar dificultades en la detección de caracteres.

In [26]:
x_train['A'] = [x_train.A.str.split(' ')[index][0].lower()
             for index in range(0, len(x_train))]

x_train['B'] = [x_train.B.str.split(' ')[index][0].lower()
             for index in range(0, len(x_train))]

x_train['Pronoun'] = [x_train.Pronoun.str.split(' ')[index][0].lower()
             for index in range(0, len(x_train))]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-doc

In [27]:
x_test['A'] = [x_test.A.str.split(' ')[index][0].lower()
             for index in range(0, len(x_test))]

x_test['B'] = [x_test.B.str.split(' ')[index][0].lower()
             for index in range(0, len(x_test))]

x_test['Pronoun'] = [x_test.Pronoun.str.split(' ')[index][0].lower()
             for index in range(0, len(x_test))]

Modificando cadena de texto a minúsculas para evitar omisiones en la búsqueda contra las columnas A y B

In [28]:
x_train['Text']=x_train['Text'].str.lower()


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [29]:
x_test['Text']=x_test['Text'].str.lower()

### Modificaciones extras

In [30]:
#borro esta linea porque me da problemas y no logro solucionarlo en los loops para sacar la posicion del pronombre
x_test=x_test.drop(x_test.index[401],)
test_labels=test_labels.drop(test_labels.index[401],)
x_test = x_test.reset_index(drop=True)
test_labels = test_labels.reset_index(drop=True)

In [31]:
#por rendimiento de mi máquina elimino las ultimas 500 lineas
x_train=x_train.drop(x_train.index[1001:1501])

In [32]:
#por rendimiento de mi máquina elimino las ultimas 500 lineas
labels=labels.drop(labels.index[1001:1501])

## *Modelo 1*

###    Justificar el porqué del modelo elejido.

Se usará un modelo de regresion lineal para medir la distancia entre las palabras. La palabra que esté "más cerca" será aquella a la que se refiere el pronombre 

In [33]:
train_df_model1=x_train
test_df_model1=x_test

In [34]:
#Eliminando features que no se usarán en train
train_df_model1=train_df_model1.drop(['Text', 'Pronoun','A', 'B'], axis=1)

#Adicionando instancias en train
train_df_model1['TO_A'] = train_df_model1['Pronoun-offset'] - train_df_model1['A-offset']
train_df_model1['TO_B'] = train_df_model1['Pronoun-offset'] - train_df_model1['B-offset']

#Eliminando features offset
train_df_model1=train_df_model1.drop(['A-offset', 'B-offset'], axis=1)


#Eliminando features que no se usarán en test
test_df_model1=test_df_model1.drop(['index','Text', 'Pronoun','A', 'B'], axis=1)

#Adicionando instancias en test
test_df_model1['TO_A'] = test_df_model1['Pronoun-offset'] - test_df_model1['A-offset']
test_df_model1['TO_B'] = test_df_model1['Pronoun-offset'] - test_df_model1['B-offset']

#Eliminando features offset
test_df_model1=test_df_model1.drop(['A-offset', 'B-offset'], axis=1)

In [35]:
train_df_model1

,Pronoun-offset,TO_A,TO_B
0,274,83,67
1,284,56,33
2,265,92,19
3,321,147,-15
4,437,218,143
...,...,...,...
996,317,39,26
997,438,125,100
998,489,184,163
999,334,139,74


In [36]:
test_df_model1

,Pronoun-offset,TO_A,TO_B
0,90,90,74
1,272,111,23
2,282,-27,-74
3,294,101,-71
4,357,81,38
...,...,...,...
494,433,178,105
495,246,135,31
496,348,89,82
497,284,139,76


In [37]:
from sklearn import  linear_model
lr_multiple = linear_model.LinearRegression()
lr_multiple.fit(train_df_model1, labels)
Y_pred_multiple = lr_multiple.predict(test_df_model1)

### Calculando la precisión del modelo

In [38]:
print('Accuracy Model 1:')
print(lr_multiple.score(train_df_model1, labels))

Accuracy Model 1:
0.011744138019593064


### Interpretar y explicar los resultados del modelo.

Como era de esperar la accuracy solo del Train ya nos da un valor muy pequeño, apenas de un 0,01. Aunque validaramos en test esta accuracy no mejoraria ustancialmente.

Esto tiene sentido porque realmente no hay una dependenca entre la distancia entre pronombre y el nombre al que hace  referencia. Cada frase es diferente y no hay una norma que nos diga a qué distancia se tiene que poner un pronombre, de hecho se puede dar el caso que el nombre más cercano al pronombre, no sea al que hace referencia.

*Entrenamiento del modelo*

In [39]:
from sklearn.linear_model import LinearRegression
modelo1 = LinearRegression()
modelo1.fit(X = train_df_model1, y=labels)

LinearRegression()

*Calculando el error medio cuadrático*

In [40]:
from sklearn.metrics import mean_squared_error
predict = modelo1.predict(X = train_df_model1)
print(predict[0:3,])

rmse = mean_squared_error(
        y_true  = labels,
        y_pred  = predict,
        squared = False
       )
print("")
print(f"El error (rmse) de test es: {rmse}")

[1.37302513 1.36055519 1.31709018]

El error (rmse) de test es: 0.6530425148334477


La exactitud del modelo de entrenamiento es considerablemente bajo, tan solo del 0,1%, por su parte el error cuadrático medio del test es del 65%. Las distancias entre pronombres y nombres parece no ser una regla general, por lo cual se observa que no hay ninguna dependencia entre ellas. El nombre más cercano a un pronombre no necesariamente hace alusión al nombre más cercano.

## ***Modelo 2***

###    Justificar el porqué del modelo elejido.

Hemos elejisdo el modelo BERT ya que es Bidireccional y  usa el self-attention en a todos los tokens de una secuencia a la vez, de manera que para el ejercicio que tenemos que hacer que es encontrar a quién hace referencia el pronombre, BERT nos va a calcular esa atención.
 

###    Entrenar el modelo.
 

Vamos a cargar el tokenizador de Bert

In [41]:
from transformers import DistilBertTokenizer, DistilBertModel
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Se usará el Tokenizador de DistilBert por cuanto es una versión destilada de BERT, es más pequeño, más rápido y más ligero. Tiene un 40 % menos de parámetros que bert-base-uncased, se ejecuta un 60 % más rápido y conserva más del 95 % del rendimiento de BERT

In [42]:
from transformers import DistilBertTokenizer, DistilBertModel
import torch

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

Creando lista de personas en columnas (A y B) para tokenizar, tanto para train como para test

In [43]:
for  i in x_train['A']:
    #print(i)
    tokenizer.add_tokens([i], special_tokens=True)

In [44]:
for  i in x_train['B']:
    tokenizer.add_tokens([i], special_tokens=True)


In [45]:
for  i in x_test['A']:
    #print(i)
    tokenizer.add_tokens([i], special_tokens=True)

In [46]:
for  i in x_test['B']:
    #print(i)
    tokenizer.add_tokens([i], special_tokens=True)

Tokenizando train y test

In [47]:
tokenized = x_train["Text"].apply((lambda x: tokenizer(x,add_special_tokens=True,return_tensors='tf')))

In [48]:
tokenized_test = x_test["Text"].apply((lambda x: tokenizer(x,add_special_tokens=True,return_tensors='tf')))

In [49]:
len(tokenized_test)

499

In [50]:
import tensorflow as tf
train_dataset =tf.data.Dataset.from_tensor_slices((dict(x_train),list(labels)))
test_dataset = tf.data.Dataset.from_tensor_slices((dict(x_test), list(test_labels)))

Cargando el modelo y cargando textos tokenizados

In [52]:
#modelo2 = DistilBertModel.from_pretrained("distilbert-base-uncased")
from transformers import DistilBertTokenizer, TFDistilBertModel
modelo2 = TFDistilBertModel.from_pretrained("distilbert-base-uncased")

Downloading:   0%|          | 0.00/347M [00:00<?, ?B/s]

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['activation_13', 'vocab_projector', 'vocab_transform', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


Dado que se han adicionado  nuevas palabras, es preciso modificar el tamaño del embeding

In [53]:
modelo2.resize_token_embeddings(len(tokenizer))

Aplicando el modelo en las cadenas de texto de train y test

In [54]:
output=[]
for i in range(len(tokenized)):
    output.append(modelo2(tokenized[i]))
    i=i+1

In [55]:
output_test=[]
for i in range(len(tokenized_test)):
    output_test.append(modelo2(tokenized_test[i]))
    i=i+1

In [56]:
#me equivoque con el range por eso quito el último
output = output[:-1]

Las salidas de cada token están contenidas en la matriz outputsdado que solo se requieren los tokens de pronombre, nombre A y nombre  B, se desarrollan funciones de búsqueda donde por cada token en la matriz output esta se conviette en una palabra indentificando la coincidencia que se busca, si esta existe, se almacenará en una lista.

In [57]:
Pronoun=[]
for i in range(len(tokenized)-1):
    for inputids in  (tokenized [i].input_ids):           
        posicioninput=0
        for token in inputids:
            tk=tokenizer.convert_ids_to_tokens([token])
            posicioninput=posicioninput+1
            for x in tk:
                tkn = str(x)
                b=x_train['Pronoun'][i]
                if tkn==b:
                    tokenizedinput=tokenized [i].input_ids[0].shape
                    inputidscontrol=inputids.shape
                    if ((inputidscontrol==tokenizedinput)and (inputids[1]==tokenized [i].input_ids[0][1])):
                        #print(i)
                        #print(inputids[1])
                        #print(tokenized [i].input_ids[0][1])
                        pos=posicioninput-1
                        Pronoun.append(pos)
                        i=i+1

In [58]:
Pronoun_test=[]
for i in range(len(tokenized_test)-1):
    for inputids in  (tokenized_test [i].input_ids):           
        posicioninput=0
        for token in inputids:
            tk=tokenizer.convert_ids_to_tokens([token])
            posicioninput=posicioninput+1
            for x in tk:
                tkn = str(x)
                b=x_test['Pronoun'][i]
                if tkn==b:
                    tokenizedinput=tokenized_test [i].input_ids[0].shape
                    inputidscontrol=inputids.shape
                    if ((inputidscontrol==tokenizedinput)and (inputids[1]==tokenized_test [i].input_ids[0][1])):
                        #print(i)
                        pos=int(posicioninput-1)
                        Pronoun_test.append(pos)
                        i=i+1



In [59]:
A=[]
for i in range(len(tokenized)-1):
    for inputids in  (tokenized [i].input_ids):           
        posicioninput=0
        for token in inputids:
            tk=tokenizer.convert_ids_to_tokens([token])
            posicioninput=posicioninput+1
            for x in tk:
                tkn = str(x)
                b=x_train['A'][i]
                if tkn==b:
                    tokenizedinput=tokenized [i].input_ids[0].shape
                    inputidscontrol=inputids.shape
                    if ((inputidscontrol==tokenizedinput)and (inputids[1]==tokenized [i].input_ids[0][1])):
                        pos=posicioninput-1
                        A.append(pos)
                        i=i+1

In [60]:
A_test=[]
for i in range(len(tokenized_test)-1):
    for inputids in  (tokenized_test [i].input_ids):           
        posicioninput=0
        for token in inputids:
            tk=tokenizer.convert_ids_to_tokens([token])
            posicioninput=posicioninput+1
            for x in tk:
                tkn = str(x)
                b=x_test['A'][i]
                if tkn==b:
                    tokenizedinput=tokenized_test [i].input_ids[0].shape
                    inputidscontrol=inputids.shape
                    if ((inputidscontrol==tokenizedinput)and (inputids[1]==tokenized_test [i].input_ids[0][1])):
                        #print(i)
                        #print(inputids[1])
                        #print(tokenized [i].input_ids[0][1])
                        pos=posicioninput-1
                        A_test.append(pos)
                        i=i+1

In [61]:
B=[]
for i in range(len(tokenized)-1):
    for inputids in  (tokenized [i].input_ids):           
        posicioninput=0
        for token in inputids:
            tk=tokenizer.convert_ids_to_tokens([token])
            posicioninput=posicioninput+1
            for x in tk:
                tkn = str(x)
                b=x_train['B'][i]
                if tkn==b:
                    tokenizedinput=tokenized [i].input_ids[0].shape
                    inputidscontrol=inputids.shape
                    if ((inputidscontrol==tokenizedinput)and (inputids[1]==tokenized [i].input_ids[0][1])):
                        pos=posicioninput-1
                        B.append(pos)
                        i=i+1


In [62]:
B_test=[]
for i in range(len(tokenized_test)-1):
    for inputids in  (tokenized_test [i].input_ids):           
        posicioninput=0
        for token in inputids:
            tk=tokenizer.convert_ids_to_tokens([token])
            posicioninput=posicioninput+1
            for x in tk:
                tkn = str(x)
                b=x_test['B'][i]
                if tkn==b:
                    tokenizedinput=tokenized_test [i].input_ids[0].shape
                    inputidscontrol=inputids.shape
                    if ((inputidscontrol==tokenizedinput)and (inputids[1]==tokenized_test [i].input_ids[0][1])):
                        #print(i)
                        #print(inputids[1])
                        #print(tokenized [i].input_ids[0][1])
                        pos=posicioninput-1
                        B_test.append(pos)
                        i=i+1

Las funciones arrojan una lista con ubicaciones. Se convierten listas en data frames agrupandolas para train y para test.

In [64]:
B = pd.DataFrame (B, columns = ['tknB'])
A = pd.DataFrame (A, columns = ['tknA'])
Pronoun = pd.DataFrame (Pronoun, columns = ['tknP'])
B_test = pd.DataFrame (B_test, columns = ['tknB'])
A_test = pd.DataFrame (A_test, columns = ['tknA'])
Pronoun_test = pd.DataFrame (Pronoun_test, columns = ['tknP'])
labels= pd.DataFrame (labels, columns = ['label'])
test_labels= pd.DataFrame (test_labels, columns = ['label'])

In [65]:
frames  = pd.concat([Pronoun,A, B], axis=1)
frames_test  = pd.concat([Pronoun_test,A_test, B_test], axis=1)

Tanto para train como para test se ubican los pronombres, los nombres A y Nombres B para concatenar los tres vectores.

In [66]:
vectors=[]
vectorpronoun=[]
vectorA=[]
vectorB=[]
for i in range(len(frames)):
  value_pronoun=frames['tknP'][i].astype(int)
  value_A=frames['tknA'][i].astype(int)
  value_B=frames['tknB'][i].astype(int)
  vectorpronoun.append(output[i].last_hidden_state[0][value_pronoun])
  vectorA.append(output[i].last_hidden_state[0][value_A])
  vectorB.append(output[i].last_hidden_state[0][value_B])
  pr=output[i].last_hidden_state[0][value_pronoun]
  A=output[i].last_hidden_state[0][value_A]
  B=output[i].last_hidden_state[0][value_B]
  vectors.append(np.concatenate((pr, A,B), axis=None))
  i=i+1

In [67]:
vectors_test=[]
vectorpronoun_test=[]
vectorA_test=[]
vectorB_test=[]
for i in range(len(frames_test)):
  value_pronoun=frames_test['tknP'][i].astype(int)
  value_A=frames_test['tknA'][i].astype(int)
  value_B=frames_test['tknB'][i].astype(int)
  vectorpronoun_test.append(output_test[i].last_hidden_state[0][value_pronoun])
  vectorA_test.append(output_test[i].last_hidden_state[0][value_A])
  vectorB_test.append(output_test[i].last_hidden_state[0][value_B])
  pr=output_test[i].last_hidden_state[0][value_pronoun]
  A=output_test[i].last_hidden_state[0][value_A]
  B=output_test[i].last_hidden_state[0][value_B]
  vectors_test.append(np.concatenate((pr, A,B), axis=None))
  i=i+1
  

Se usará una Regresion lineal para poder comparar esta salida con los resultados del primer modelo.

In [68]:
from sklearn import  linear_model
lr_multiple = linear_model.LinearRegression()


Eliminando última línea para ajustar el range

In [ ]:
labels=labels[:-1]

In [69]:
lr_multiple.fit(vectors, labels)

LinearRegression()

In [70]:
Y_pred_test = lr_multiple.predict(vectors_test)

Eliminando última línea para ajustar el range

In [72]:
test_labels=test_labels[:-1]

In [74]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt

print("Error cuadrático médio: %.2f" % mean_squared_error(test_labels, Y_pred_test))
print("Coeficiente de determinación: %.2f" % r2_score(test_labels, Y_pred_test))

Error cuadrático médio: 0.89
Coeficiente de determination: -1.03


###    * Calculando la accuracy del modelo


In [76]:
print('Accuracy del modelo:')
print(lr_multiple.score(vectors, labels))

Accuracy del modelo:
0.9999999999148771


###    * Interpretando resultados  

Usando el mismo modelo, pero con los pesos calculados segun la Importancia con DestilBERT, la exactitud en train pasa al 99%. El error cuadrático medio respecto del test arroja un 89%, es decir que la aproximación del modelo es basante aceptable.
